In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

CREATING WIKIPEDIA TOOL

In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002C4F6A4CDF0>)

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [7]:
retriever_tool.name

'langsmith_search'

In [8]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools=[wiki,arxiv,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\subha\\OneDrive - The University of Alabama\\Research\\PYTHON_ALLCOURSES_JOB\\NLP_LLM_Coursera_Misc\\LANGCHAIN\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 

In [11]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [13]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [15]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [25]:
agent_executor.invoke({"input":"What is the difference between langsmith and langchain in wikipedia"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `LangSmith`


Page: Lang Lang discography
Summary: The discography of classical pianist Lang Lang includes nine studio albums, four live albums, one single, one compilation, two soundtracks, and three contributions
Invoking: `wikipedia` with `LangChain`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-casesLangSmith is related to the discography of classical pianist Lang Lang, while LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). LangChain serves as a language model integration framework for various use-cases.

> Finished chain.


{'input': 'What is the difference between langsmith and langchain in wikipedia',
 'output': 'LangSmith is related to the discography of classical pianist Lang Lang, while LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). LangChain serves as a language model integration framework for various use-cases.'}

In [23]:
agent_executor.invoke({"input":"Give me four papers on signal processing in 2022 and 2023?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'signal processing 2022'}`


Published: 2023-03-10
Title: To further understand graph signals
Authors: Feng Ji, Wee Peng Tay
Summary: Graph signal processing (GSP) is a framework to analyze and process
graph-structured data. Many
Invoking: `arxiv` with `{'query': 'signal processing 2023'}`


Published: 2023-02-24
Title: Graph signal processing with categorical perspective
Authors: Feng Ji, Xingchao Jian, Wee Peng Tay
Summary: In this paper, we propose a framework for graph signal processiI found two papers on signal processing in 2023:

1. **Title:** To further understand graph signals
   **Authors:** Feng Ji, Wee Peng Tay
   **Published:** 2023-03-10
   **Summary:** Graph signal processing (GSP) is a framework to analyze and process graph-structured data.

2. **Title:** Graph signal processing with categorical perspective
   **Authors:** Feng Ji, Xingchao Jian, Wee Peng Tay
   **Published:** 2023-02-24
   **Sum

{'input': 'Give me four papers on signal processing in 2022 and 2023?',
 'output': 'I found two papers on signal processing in 2023:\n\n1. **Title:** To further understand graph signals\n   **Authors:** Feng Ji, Wee Peng Tay\n   **Published:** 2023-03-10\n   **Summary:** Graph signal processing (GSP) is a framework to analyze and process graph-structured data.\n\n2. **Title:** Graph signal processing with categorical perspective\n   **Authors:** Feng Ji, Xingchao Jian, Wee Peng Tay\n   **Published:** 2023-02-24\n   **Summary:** This paper proposes a framework for graph signal processing with a categorical perspective.'}